# Importing the Required Dependencies

In [31]:
# Import Python Standard Library dependencies

import os
from functools import partial
from pathlib import Path
from typing import Tuple, List, Dict, Optional
from collections import OrderedDict
import xml.etree.ElementTree as ET
from typing import Any, Dict, Optional

# Import utility functions
from cjm_pil_utils.core import get_img_files
from cjm_psl_utils.core import download_file, file_extract
from cjm_pytorch_utils.core import pil_to_tensor, tensor_to_pil, get_torch_device, set_seed, denorm_img_tensor, move_data_to_device
from cjm_torchvision_tfms.core import ResizeMax, PadSquare, CustomRandomIoUCrop

# Import the distinctipy module
from distinctipy import distinctipy

# Import matplotlib for creating plots
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

# Import the pandas package
import pandas as pd

# Do not truncate the contents of cells and display all rows and columns
pd.set_option('max_colwidth', None, 'display.max_rows', None, 'display.max_columns', None)

# Import PIL for image manipulation
from PIL import Image

# Import PyTorch dependencies
import torch
from torch import Tensor
from torch.utils.data import Dataset, DataLoader

# Import torchvision dependencies
import torchvision
torchvision.disable_beta_transforms_warning()
from torchvision.tv_tensors import BoundingBoxes
from torchvision.utils import draw_bounding_boxes
import torchvision.transforms.v2  as transforms
from torchvision.models.detection.roi_heads import fastrcnn_loss
from torchvision.models.detection.rpn import concat_box_prediction_layers
from torch.utils.tensorboard import SummaryWriter
from torch.amp import autocast

# Import tqdm for progress bar
from tqdm.auto import tqdm
from engine import train_one_epoch, evaluate
import utils

In [32]:
# 데이터 세트 저장 경로 정의
dataset_path =Path("./validsets/")
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weight='DEFAULT', max_size=1920)

### CVAT 주석을 파싱하기위한 함수 정의(Define a function to parse the CVAT XML annotations)

In [33]:
def parse_cvat_bbox_xml(xml_content):
    """
    CVAT 바운딩 박스 주석 파일의 주어진 XML 콘텐츠를 구문 분석하고
    판다 데이터프레임으로 변환합니다.

    이 함수는 XML 콘텐츠를 처리하여 각 이미지와 관련 바운딩 박스에 대한 정보를 추출합니다.
    이미지 및 관련 바운딩 박스에 대한 정보를 추출합니다. 그런 다음 데이터는
    데이터 프레임으로 구조화하여 조작과 분석이 용이하도록 합니다.

    Parameters:
    xml_content (str): A string containing the XML content from a CVAT bounding box
                       annotation file.

    Returns:
    pandas.DataFrame: A DataFrame where each row represents an image and contains
                      the following columns:
                      - Image ID (int): The unique identifier of the image.
                      - Image Name (str): The name of the image.
                      - Width (int): The width of the image in pixels.
                      - Height (int): The height of the image in pixels.
                      - Boxes (list): A list of dictionaries, where each dictionary
                                      represents a bounding box with keys 'Label',
                                      'xtl', 'ytl', 'xbr', and 'ybr' indicating the
                                      label and coordinates of the box.
    """

    # XML 콘텐츠 구문 분석
    root = ET.fromstring(xml_content)
    data = {}

    # XML의 각 이미지 요소를 반복합니다.
    for image in root.findall('image'):
        # Extract image attributes
        image_id = image.get('id')
        image_name = image.get('name')
        width = image.get('width')
        height = image.get('height')

        # 이미지 데이터를 저장할 사전을 초기화합니다.
        image_data = {
            'Image ID': int(image_id),
            'Image Name': image_name,
            'Width': int(width),
            'Height': int(height),
            'Boxes': []
        }

        # 이미지 내의 각 바운딩 박스 요소를 반복합니다.
        for box in image.findall('box'):
            # Extract box attributes
            label = box.get('label')
            xtl = float(box.get('xtl'))
            ytl = float(box.get('ytl'))
            xbr = float(box.get('xbr'))
            ybr = float(box.get('ybr'))

            # 상자 데이터에 대한 사전을 구축합니다.
            box_data = {
                'Label': label,
                'xtl': xtl,
                'ytl': ytl,
                'xbr': xbr,
                'ybr': ybr
            }

            # 이미지의 '박스' 목록에 박스 데이터를 추가합니다.
            image_data['Boxes'].append(box_data)

        # 이미지 데이터를 데이터 사전의 ID에 매핑합니다.
        data[image_id] = image_data

    # 데이터 사전을 데이터 프레임으로 변환하고 반환합니다.
    return pd.DataFrame.from_dict(data, orient='index')

In [34]:
import os

# 이미지 리스팅
# img_dir = dataset_path/'images'
img_dir = './3_1'
img_dict = {
    file.stem : file
    for file in get_img_files(img_dir)
}

# annotation 리스팅
annotation_dir_path = './annotation'
dir_list = os.listdir(annotation_dir_path)

start = True
for xml_file in dir_list:
    print(f'{annotation_dir_path}/{xml_file}')
    if start:
         with open(f'{annotation_dir_path}/{xml_file}', 'r', encoding='utf-8') as file:
            xml_content = file.read()
            annotation_df = parse_cvat_bbox_xml(xml_content)
            start = False
    else:
        with open(f'{annotation_dir_path}/{xml_file}', 'r', encoding='utf-8') as file:
            xml_content = file.read()
            annotation_df = pd.concat([annotation_df, parse_cvat_bbox_xml(xml_content)], ignore_index=True)

annotation_df['Image ID'] = annotation_df['Image Name'].apply(lambda x: x.split('.png')[0])


annotation_df = annotation_df.set_index('Image ID') # 새 '이미지 ID' 열을 데이터프레임의 인덱스로 설정
boxes_df = annotation_df['Boxes'].explode().to_frame().Boxes.apply(pd.Series)

class_names = boxes_df['Label'].unique().tolist() # 'annotation_df' 데이터프레임에서 고유 레이블 목록 가져오기

class_to_idx = {c: i for i, c in enumerate(class_names)}

label_dic = {}
label_dic['car']=1
label_dic['truck']=2
label_dic['bus']=3

./annotation/Cheonan_gangjeong_20201019_1600_MON_15m_RH_highway_TW3_sunny_FHD.xml
./annotation/Cheonan_gangjeong_20201019_1700_MON_15m_NH_highway_TW3_sunny_FHD.xml


In [35]:
def make_prediction(preds, threshold):
    
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]
        
    return preds

def makeBox(image_raw,bbox,objects):
  image = image_raw.copy()
  for i in range(len(objects)):
    cv2.rectangle(image,(int(bbox[i][0]),int(bbox[i][1])),(int(bbox[i][2]),int(bbox[i][3])),color = (0,255,0),thickness = 1)
    cv2.putText(image, objects[i], (int(bbox[i][0]), int(bbox[i][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2) # 크기, 색, 굵기
  return image

def make_info_txt(file_name, object_name, bbox, mode, scores = None):
    file_name = str(file_name)    
    file_name = file_name.split("\\")[-1]
    if mode == 'gt':
        print(bbox)
        boxx = []
        labels = []
        for item in range(len(bbox)):
            box = [bbox[item]['xtl']]
            box.append(bbox[item]['ytl'])
            box.append(bbox[item]['xbr'])
            box.append(bbox[item]['ybr'])
            boxx.append(box)
            labels.append(bbox[item]['Label'])


        with open("validsets/mAP/input/ground-truth/{}.txt".format(file_name[:-4]),"w") as f:
            for i in range(len(labels)):
                f.write("{} ".format(labels[i])+" ".join(map(str,map(int,boxx[i])))+"\n")
    
    if mode == 'rt':
        assert scores != None
        with open("validsets/mAP/input/detection-results/{}.txt".format(file_name[:-4]),"w") as f:
            for i in range(len(object_name)):
                f.write("{} ".format(object_name[i])+"{} ".format(scores[i])+" ".join(map(str,map(int,bbox[i])))+"\n")
import cv2
def evaluation(img_dict, annotation_df, new = False):
    # 폴더생성
    if new == True:
        print("Clear Directory")
        filelist = [ f for f in os.listdir("./validsets/mAP/input/ground-truth/")]
        for f in filelist:
            os.remove(os.path.join("./validsets/mAP/input/ground-truth/", f))     
        filelist = [ f for f in os.listdir("./validsets/mAP/input/detection-results/")]
        for f in filelist:
            os.remove(os.path.join("./validsets/mAP/input/detection-results/", f)) 

    # 데이터 파싱 후 평가
    img_key = list(img_dict.keys())
    for i in range(len(img_key)):
        img_id = img_key[i]
        img_name = img_dict[img_id]
        img_path = f"{img_name}"
        img = Image.open(img_path).convert("RGB")
        img = img.resize((1920, 1080))
        to_tensor = torchvision.transforms.ToTensor()
        test_image = to_tensor(img).unsqueeze(0)
        model.eval()
        with torch.no_grad():
            prediction = model(test_image.to(device))

        prediction=make_prediction(prediction, 0.8)

        boxes = prediction[0]['boxes']
        labels = prediction[0]['labels']
        scores = prediction[0]['scores']

        object_name = []
        for lb in labels:
            object_name.append([k for k, v in label_dic.items() if v == lb][0])
        make_info_txt(img_name, object_name, boxes, mode='rt', scores = scores)
        make_info_txt(img_name, object_name, annotation_df.loc[img_id, 'Boxes'], mode='gt')
        img = np.array(Image.open(img_path).convert("RGB"))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB ) 

        im = makeBox(img,boxes,object_name)
        
        cv2.imshow("image",im)
        cv2.waitKey(1)

    print("Result")
    f = os.popen("python /dataset/mAP/main.py --no-animation ") #openCV작동 안해서 --no-animation
    print(f.read())

In [36]:
def evaluation2(img_dict):
    # 폴더생성
    if new == True:
        print("Clear Directory")
        filelist = [ f for f in os.listdir("./validsets/mAP/input/ground-truth/")]
        for f in filelist:
            os.remove(os.path.join("./validsets/mAP/input/ground-truth/", f))     
        filelist = [ f for f in os.listdir("./validsets/mAP/input/detection-results/")]
        for f in filelist:
            os.remove(os.path.join("./validsets/mAP/input/detection-results/", f)) 

    # 데이터 파싱 후 평가
    img_key = list(img_dict.keys())
    for i in range(len(img_key)):
        img_id = img_key[i]
        img_name = img_dict[img_id]
        img_path = f"{img_name}"
        img = Image.open(img_path).convert("RGB")
        img = img.resize((1920, 1080))
        to_tensor = torchvision.transforms.ToTensor()
        test_image = to_tensor(img).unsqueeze(0)
        model.eval()
        with torch.no_grad():
            prediction = model(test_image.to(device))

        prediction=make_prediction(prediction, 0.5)

        boxes = prediction[0]['boxes']
        labels = prediction[0]['labels']
        scores = prediction[0]['scores']

        object_name = []
        for lb in labels:
            object_name.append([k for k, v in label_dic.items() if v == lb][0])
        make_info_txt(img_name, object_name, boxes, mode='rt', scores = scores)
        make_info_txt(img_name, object_name, annotation_df.loc[img_id, 'Boxes'], mode='gt')
        img = np.array(Image.open(img_path).convert("RGB"))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB ) 

        im = makeBox(img,boxes,object_name)
        
        cv2.imshow("image",im)
        cv2.waitKey(2)

    print("Result")
    # f = os.popen("python /dataset/mAP/main.py --no-animation ") #openCV작동 안해서 --no-animation
    print(f.read())

In [37]:

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights='DEFAULT', max_size=1920)

num_classes = 4
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.to(device)
model.load_state_dict(torch.load("fasterrcnn_resnet50_fpn_v2.pth",map_location=device)) #epoch지정

<All keys matched successfully>

In [38]:
evaluation(img_dict, annotation_df, new = True)
import time
time.sleep(5)
cv2.destroyAllWindows()

Clear Directory
[{'Label': 'car', 'xtl': 1187.0, 'ytl': 755.0, 'xbr': 1296.0, 'ybr': 843.0}, {'Label': 'truck', 'xtl': 418.0, 'ytl': 363.0, 'xbr': 469.0, 'ybr': 435.0}, {'Label': 'truck', 'xtl': 1185.0, 'ytl': 597.0, 'xbr': 1362.0, 'ybr': 778.0}, {'Label': 'truck', 'xtl': 1380.0, 'ytl': 709.0, 'xbr': 1667.0, 'ybr': 936.0}, {'Label': 'truck', 'xtl': 851.0, 'ytl': 565.0, 'xbr': 909.0, 'ybr': 621.0}, {'Label': 'truck', 'xtl': 882.0, 'ytl': 438.0, 'xbr': 976.0, 'ybr': 551.0}, {'Label': 'truck', 'xtl': 384.0, 'ytl': 273.0, 'xbr': 414.0, 'ybr': 311.0}, {'Label': 'truck', 'xtl': 677.0, 'ytl': 333.0, 'xbr': 727.0, 'ybr': 390.0}, {'Label': 'truck', 'xtl': 718.0, 'ytl': 362.0, 'xbr': 767.0, 'ybr': 418.0}, {'Label': 'bus', 'xtl': 1503.0, 'ytl': 928.0, 'xbr': 1813.0, 'ybr': 1077.0}, {'Label': 'car', 'xtl': 692.0, 'ytl': 560.0, 'xbr': 752.0, 'ybr': 611.0}, {'Label': 'car', 'xtl': 1049.0, 'ytl': 659.0, 'xbr': 1120.0, 'ybr': 713.0}, {'Label': 'car', 'xtl': 550.0, 'ytl': 512.0, 'xbr': 596.0, 'ybr': 55